In [4]:
import os, glob, csv, statistics, math, numpy as np, exifread, geopandas as gpd, pandas as pd, cameratransform as ct
from osgeo import gdal, osr
from shapely.geometry import Point, Polygon
from PIL import Image, ExifTags

# set the base directory where everything lives
base_dir = r"C:\Users\gdlarsen\Documents\UAS_survey-JHI_WE_S2\F1"
out_dir = base_dir

# set the subdirectories where the images (img) and flight log (flog) live
img_dir = f"{base_dir}\\OUTPUTS"
flog_dir = f"{base_dir}\\FLIGHT RECORD"

# set the subdirectories where the georeferenced images will be output and where the footprints feature will be output
imgout_dir = f"{out_dir}\\Georeferenced"
shpout_dir = f"{out_dir}\\Shapefiles"

# create output directories, if necessary
os.makedirs(shpout_dir, exist_ok = True)
os.makedirs(imgout_dir, exist_ok = True)

# Look up the sensor width (mm) of the camera, if you can find it: https://www.dxomark.com/
# If you can't find it, use 'unknown', for example: sensor_width = 'unknown'

#This is the sensor width for the Sony a6100
sensor_width = 23.5

In [5]:
# This function scrapes chunks of metadata for the Sony a6100
# it's the most I could find using native python methods
# (i.e. not calling ExifTool or anything else in the background)
# could more simply and robustly recode this section to just call and import from exiftool
# might be improved with more reading about EXIF structure and targeted tag IDs 
# https://exiftool.org/TagNames/EXIF.html

def scrape_metadata(img):
    metadata_package = {}
    with Image.open(img) as im:
        exif = im.getexif()
        exif_data = exif.get_ifd(0x8769)
        metadata_chunk1={}
        for tag_id in exif_data:
            tag = ExifTags.TAGS.get(tag_id, tag_id)
            content = [exif_data.get(tag_id)]
            if type(content).__name__ == 'IFDRational':
                content = float(content)
            metadata_chunk1[tag] = content
        FunctionExif = "b'0231'"
        if str(metadata_chunk1['ExifVersion'][0]) != str(FunctionExif):
            print(f"Warning: this function was designed for metadata produced in {FunctionExif} format, not {metadata_chunk1['ExifVersion'][0]}. Proceed with caution, metadata could be broken.")
        del metadata_chunk1['MakerNote'], metadata_chunk1['FlashPixVersion'], metadata_chunk1['FileSource'], metadata_chunk1['SceneType'], metadata_chunk1['ComponentsConfiguration']
        if sensor_width == 'unknown':
            metadata_chunk1['SensorWidth'] = [float(36*metadata_chunk1['FocalLength'][0]/metadata_chunk1['FocalLengthIn35mmFilm'][0])]
        else:
            metadata_chunk1['SensorWidth'] = [sensor_width]
        metadata_chunk1['FieldOfView'] = [math.degrees(2*math.atan(metadata_chunk1['SensorWidth'][0]/(2*metadata_chunk1['FocalLength'][0])))]
        metadata_chunk1['AspectRatio'] = [metadata_chunk1['ExifImageWidth'][0]/metadata_chunk1['ExifImageHeight'][0]]
        metadata_package.update(metadata_chunk1)
        
        metadata_chunk2 = {}
        for tag_id in ExifTags.IFD:
            ifd = exif.get_ifd(tag_id)
            if tag_id == ExifTags.IFD.GPSInfo:
                resolve = ExifTags.GPSTAGS
                for k, v in ifd.items():
                    tag = resolve.get(k, k)
                    metadata_chunk2[tag] = [v]
        del metadata_chunk2['GPSVersionID'], metadata_chunk2['GPSAltitudeRef']
        metadata_chunk2['GPSLongitudeDD'] = [(float(metadata_chunk2['GPSLongitude'][0][0]) + float(metadata_chunk2['GPSLongitude'][0][1])/60 + float(metadata_chunk2['GPSLongitude'][0][2])/(60*60))*(-1 if metadata_chunk2['GPSLongitudeRef'][0] == 'W' else 1)]
        metadata_chunk2['GPSLatitudeDD'] = [(float(metadata_chunk2['GPSLatitude'][0][0]) + float(metadata_chunk2['GPSLatitude'][0][1])/60 + float(metadata_chunk2['GPSLatitude'][0][2])/(60*60))*(-1 if metadata_chunk2['GPSLatitudeRef'][0] == 'S' else 1)]
        metadata_package.update(metadata_chunk2)

        xmpdata = im.getxmp()['xmpmeta']['RDF']['Description']
        metadata_chunk3 = xmpdata
        metadata_package.update(metadata_chunk3)

        FunctionCamera = 'ILCE-6100 v1.00'
        if str(metadata_chunk3['CreatorTool']) != str(FunctionCamera):
            print(f"Warning: this function was designed for metadata produced by {FunctionCamera}, not {metadata_chunk3['CreatorTool']}. Proceed with caution, metadata could be broken.")
    return metadata_package

In [6]:
# This section reads the jpgs and extracts the metadata into a table, using the previous function
img_list = glob.glob(f"{base_dir}//*OUTPUT//*.jpg", recursive=True)

# img_list = img_list[355:365] # for demo

df = pd.DataFrame()
for img in img_list:
    image_name = img.split("\\")[-1].split(".")[0]
    metadata = scrape_metadata(img)
    sensor_size = float(metadata['SensorWidth'][0]), float(metadata['SensorWidth'][0]/metadata['AspectRatio'][0])
    metadict = {'ImageName': [image_name], 'ImagePath': [img]}|metadata|{'SensorSize': [sensor_size]}
    df = pd.concat([df, pd.DataFrame.from_dict(metadict)])
df = df.reset_index()

# This section corrects the camera time for timezone/offsets
from datetime import datetime as dt

if df['OffsetTime'].loc[0][0] == "+":
    offset_subtraction = True
elif df['OffsetTime'].loc[0][0] == "-":
    offset_subtraction = True
else:
    print(f"might need to code for an unsigned offset, value {df['OffsetTime'].loc[0][0]}")

# This code assumes that the time offset is consistent across the flight (reasonable assumption)
# If it varies we can recode a vectorized solution.
hoffset, moffset = df['OffsetTime'].loc[0][1:].split(":")
time_mod = pd.to_datetime(df['DateTimeOriginal'], format = "%Y:%m:%d %H:%M:%S")
if offset_subtraction:
    time_mod = time_mod - (pd.offsets.Hour(int(hoffset)) + pd.offsets.Minute(int(moffset)))
else:
    time_mod = time_mod + (pd.offsets.Hour(int(hoffset)) + pd.offsets.Minute(int(moffset)))
df['DateTime(UTC)'] = time_mod
print("Table created")

Table created


In [7]:
# Altitude is often unreliable metadata, whether from GPS or barometer.
# In this case, from inspecting photos, it's clear that the drone
# was closer to the water than GPS measurements represent
# so we're merging the drone flight log (based on time) so we can pull the drone altitude

# read in the CSV flight log
csv_list = glob.glob(f"{base_dir}//FLIGHT RECORD//*.csv", recursive=True)
ulog_list = glob.glob(f"{base_dir}//FLIGHT RECORD//*.ulg", recursive=True)
print(f"Reading log file {csv_list[0]}")
csv_df = pd.read_csv(csv_list[0])

altitude_offset = []
# format and merge the CSV flight log into our dataframe
csv_df = csv_df.rename(columns={'time(epoch)':'DateTime(Epoch)', 'time(UTC)':'DateTime(UTC)', 'lat':'DroneLatitude', 'lon':'DroneLongitude', 'alt':'DroneAltitude'})
csv_df['DateTime(UTC)']=pd.to_datetime(csv_df['DateTime(UTC)'], format = "%Y-%m-%d %H:%M:%S")
new_df = pd.merge(df, csv_df, on = 'DateTime(UTC)')
if len(df) != len(new_df):
    altitude_offset = statistics.mode(csv_df['DroneAltitude']) - statistics.mode([float(i) for i in df['GPSAltitude']])
    print(f"merged table is a different length than original table, defaulting to GPSAltitude with an offset of {altitude_offset} m")
    df['DroneAltitude'] = "NA"
    print("Drone altitude column populated with NAs")
    # this might happen if not all camera timestamps occur among drone timestamps
else:
    df = new_df
    print("Drone altitude merged with table")

Reading log file C:\Users\gdlarsen\Documents\UAS_survey-JHI_WE_S2\F1//FLIGHT RECORD\JHI_120624_ext_S2_F1 Flight 01.csv
Drone altitude merged with table


In [45]:
# This section reads uses the metadata to generate GCPs for image projection

df2 = pd.DataFrame()

for index, metadata in df.iterrows():
    # queue up important parameters from the metadata
    f = metadata['FocalLength']
    sensor_size = metadata['SensorWidth'], metadata['SensorWidth']/metadata['AspectRatio']
    image_size = metadata['ExifImageWidth'], metadata['ExifImageHeight']
    lat, lon = metadata['GPSLatitudeDD'], metadata['GPSLongitudeDD']
    if not altitude_offset:
        alt = float(metadata['DroneAltitude'])
    else:
        alt = float(metadata['GPSAltitude']) + altitude_offset
    yaw, pitch, roll = float(metadata['Yaw']), float(metadata['Pitch']), float(metadata['Roll'])

    # use cameratransform package to project image based on parameters
    cam = ct.Camera(ct.RectilinearProjection(focallength_mm = f, sensor = sensor_size, image = image_size),
                    ct.SpatialOrientation(elevation_m = alt, tilt_deg = pitch, roll_deg = roll, heading_deg = yaw, 
                                        pos_x_m = 0, pos_y_m = 0))
    
    # use cameratransform package to assign spatial values to image locations
    cam.setGPSpos(lat, lon, alt)

    img_edgepoints = ([0, 0], # top left
                      [img_w-1, 0], # top right
                      [img_w-1, img_h-1], # bottom right
                      [0, img_h-1], # bottom left
                 
                      [(img_w-1)/2, 0], # top midpoint
                      [img_w-1, (img_h-1)/2], # right midpoint
                      [(img_w-1)/2, img_h-1], # bottom midpoint
                      [0, (img_h-1)/2] # left midpoint
                     )
    GCP_list = [cam.gpsFromImage(i).tolist() + i for i in img_edgepoints]

    metadict = {'ImageName': metadata['ImageName'], 'GCPList': [GCP_list], 'geometry': [Polygon([i[0:2][::-1] for i in GCP_list][0:4])]}
    df2 = pd.concat([df2, pd.DataFrame.from_dict(metadict)])

gdf = gpd.GeoDataFrame(pd.merge(df, df2, on='ImageName'), crs="EPSG:4326")
print("GCPs ready")

GCPs ready


In [42]:
print([i[0:2] for i in GCP_list])
print([i[::-1] for i in coords])

[[58.85624240154075, -137.0952535893452], [58.85580910836961, -137.09415454431812], [58.855419167501985, -137.09470616860912], [58.85586401149918, -137.0958344306931], [58.85602576220367, -137.0947040788201], [58.8556166991308, -137.0944267352741], [58.85564159714027, -137.09527031228575], [58.85605569174655, -137.09554019720008]]
[[-137.0952535893452, 58.85624240154075], [-137.09415454431812, 58.85580910836961], [-137.09470616860912, 58.855419167501985], [-137.0958344306931, 58.85586401149918]]


In [46]:
# This section outputs the geodataframe to a shapefile with all metadata as attributes.
# Note that attribute field names get truncated to 10 characters

# This section suppresses those warnings
import warnings
warnings.filterwarnings("ignore", message="Normalized/laundered field name")

out_path = f"{shpout_dir}\\Image_footprints.shp"
gdf.to_file(out_path)

C:\Users\gdlarsen\AppData\Local\Temp\1\ipykernel_43036\1507117850.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(out_path)
C:\Users\gdlarsen\AppData\Local\miniforge3\envs\spatial\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field DateTime(U create as date field, though DateTime requested.
  ogr_write(
C:\Users\gdlarsen\AppData\Local\miniforge3\envs\spatial\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value '[[58.86442609712705, -137.06432625188143, 0.0, 0, 0], [58.864872686516996, -137.0654096120216, 0.0, 5999, 0], [58.86524247056331, -137.06483158403734, 0.0, 5999, 3999], [58.864799870942036, -137.06375783177276, 0.0, 0, 3999], [58.86464939891545, -137.06486794291536, 0.0, 2999.5, 0], [58.86505840533121, -137.06511930767905, 0.0, 5999, 1999.5], [58.865021177720244, -137.064294718675, 0.0, 2999.5, 3999], [58.86461381977653, -137.06404077287345, 0.0, 0, 1999.5]]' of field GCPList has been truncate

In [147]:
# This section reads in each image and its GCPs, references and warps the image, then saves it out as a JPEG
os.makedirs(imgout_dir, exist_ok = True)

# alternative format "GeoTIFF" is much larger per file
out_format = "JPEG"

gdal.UseExceptions()

for index, record in gdf.iterrows():

    # Read in the image file:
    ds = gdal.Open(record['ImagePath'])
    if ds is None:
        print(f"Could not open image: {record['ImagePath']}")
    # Set spatial reference:
    sr = osr.SpatialReference()
    sr.ImportFromEPSG(4326)

    # Import GCPs - there's got to be a better way to unpack i, but lists get rejected.
    gcps = [gdal.GCP(i[0], i[1], i[2], i[3], i[4]) for i in record['GCPList']]

    # Apply the GCPs to the open output file then warp it
    ds.SetGCPs(gcps, sr.ExportToWkt())

    if out_format=="JPEG":
        kwargs = {'format': 'JPEG', 'polynomialOrder':2, 'srcNodata': '0,0,0', 'dstNodata': 'nodata'}
        ds = gdal.Warp(f"{imgout_dir}\\{record['ImageName']}_GeoRef.jpg", ds, **kwargs)
    elif out_format=="GeoTIFF":
        kwargs = {'format': 'GTiff', 'polynomialOrder':2, 'srcNodata': '0,0,0', 'dstNodata': 'nodata'}
        ds = gdal.Warp(f"{imgout_dir}\\{record['ImageName']}_GeoRef.tif", ds, **kwargs)
    else:
        print(fr"format '{out_format}' not currently supported, code it yourself from https://gdal.org/en/latest/drivers/raster/index.html")
        break
    # Clear the variable (not sure if necessary, but good form)
    ds = None
    # counter, just because it can be a long process
    if (index+1) % 10 == 0:
        print(f"Completed file {index+1} out of {len(df)+1}")
print("finished processing images")

Completed file 150 out of 1685
Completed file 160 out of 1685
Completed file 170 out of 1685
Completed file 180 out of 1685
Completed file 190 out of 1685
Completed file 200 out of 1685
Completed file 210 out of 1685
Completed file 220 out of 1685
Completed file 230 out of 1685
Completed file 240 out of 1685
Completed file 250 out of 1685
Completed file 260 out of 1685
Completed file 270 out of 1685
Completed file 280 out of 1685
Completed file 290 out of 1685
Completed file 300 out of 1685
Completed file 310 out of 1685
Completed file 320 out of 1685
Completed file 330 out of 1685
Completed file 340 out of 1685
Completed file 350 out of 1685
Completed file 360 out of 1685
Completed file 370 out of 1685
Completed file 380 out of 1685
Completed file 390 out of 1685
Completed file 400 out of 1685
Completed file 410 out of 1685
Completed file 420 out of 1685
Completed file 430 out of 1685
Completed file 440 out of 1685
Completed file 450 out of 1685
Completed file 460 out of 1685
Complete